In [11]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')


# NLTK modules
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

import re

from gensim.models import Word2Vec # Word2Vec module

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer, TfidfTransformer
from sklearn.preprocessing import LabelEncoder, Normalizer
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# 1. Loading dataset

In [12]:
df = pd.read_csv('/Users/maurelco/Developer/Python/Projet 4/data/Cleaned/df_process_text_3.csv')

In [13]:
# print sample data
df.head().append(df.tail())

,Title,Body,Tags,_clean_tags,_len_body,Body_2,_len_body_2,Body_3,Title_2
0,giving unix process exclusive rw access directory,way sandbox linux process certain directory gi...,linux ubuntu process sandbox selinux,"['linux', 'ubuntu', 'process', 'sandbox', 'sel...",526,sandbox linux process certain directory give p...,462,sandbox linux process certain directory proces...,giving unix process exclusive rw access directory
1,automatic repaint minimizing window,jframe two panel one panel draw line working m...,java graphics jframe jpanel paint,"['java', 'graphics', 'jframe', 'jpanel', 'paint']",2969,jframe two panel panel draw line working minim...,2855,jframe panel panel draw line minimized window ...,automatic repaint minimizing window
2,man-in-the-middle attack security threat ssh a...,expert network security pardon question smart ...,security ssh ssh-keys openssh man-in-the-middle,"['security', 'ssh', 'ssh-keys', 'openssh', 'ma...",447,expert network security pardon smart automatin...,414,expert network security pardon smart automatin...,man-in-the-middle attack security threat ssh a...
3,managing data access simple winforms app,simple winforms data entry app us sqlite alway...,c# winforms sqlite datatable sqlconnection,"['c#', 'winforms', 'sqlite', 'datatable', 'sql...",2537,simple winforms entry app u sqlite always sing...,2382,winforms entry app sqlite always single-user a...,managing data access winforms app
4,render basic html view,basic node.js app trying get ground using expr...,javascript html node.js mongodb express,"['javascript', 'html', 'node.js', 'mongodb', '...",335,basic node.js app get ground express framework...,292,basic node.js app get ground express framework...,render basic html view
49995,bypass vertica error execution time exceeded r...,using ssis tool ole db downloading data vertic...,sql-server ssas oledb sql-server-data-tools ve...,"['sql-server', 'ssas', 'oledb', 'sql-server-da...",206,ssis tool ole db downloading vertica database ...,186,ssis tool ole db downloading vertica database ...,bypass vertica error execution time exceeded r...
49996,conflicting conditional operation currently pr...,using f uploading file already created bucket ...,python amazon-web-services amazon-s boto pytho...,"['python', 'amazon-web-services', 'amazon-s', ...",301,f uploading file already created bucket deleti...,268,uploading file bucket deleting bucket executio...,conflicting conditional operation progress bucket
49997,problem lr_find pytorch fastai course,following jupyter notebook course hit upon err...,python machine-learning deep-learning pytorch ...,"['python', 'machine-learning', 'deep-learning'...",908,jupyter notebook course hit upon line cnn_lear...,781,jupyter notebook course hit upon line cnn_lear...,problem lr_find pytorch fastai course
49998,jsonpatch escape slash jsonpatch+json,json wanted update field process my-process po...,java json rest json-patch http-patch,"['java', 'json', 'rest', 'json-patch', 'http-p...",645,json wanted update field process my-process po...,566,json wanted field process my-process pod some-...,jsonpatch escape slash jsonpatch+json
49999,org.gradle .internal .resolve .artifactnotfoun...,using version firebase google play service bui...,android android-studio android-gradle-plugin g...,"['android', 'android-studio', 'android-gradle-...",27209,version firebase google play service build sta...,27143,version firebase google play service build sta...,org.gradle .resolve .artifactnotfoundexception...


In [14]:
df = df.dropna()

In [15]:
df.isna().sum()

Title          0
Body           0
Tags           0
_clean_tags    0
_len_body      0
Body_2         0
_len_body_2    0
Body_3         0
Title_2        0
dtype: int64

In [16]:
df['body_and_title_2'] = df['Body_3'] + " " +df['Title_2']
df['body_and_title_2'].iloc[0]

'sandbox linux process certain directory process exclusive rw access dir temporary directory e.g python scripting tool directory limiting much functionality process access directory except superusers course sandbox web service basically allows user arbitrary code authorization software process linux user user harm system temporary private directory file protected user webservice giving unix process exclusive rw access directory'

In [8]:
train, test = train_test_split(df, test_size=0.3, random_state=42)

In [9]:
tag_words = pd.read_csv('/Users/maurelco/Developer/Python/Projet 4/data/Cleaned/tag_words.csv')
tag_words

,words
0,c#
1,java
2,javascript
3,python
4,asp.net
...,...
18038,cloudbees
18039,djcelery
18040,copywithzone
18041,time-precision


In [ ]:
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()

y_enc_train, labels = labelencoder(tag_words)

# Split from the loaded dataset
X_train, X_test, y_train, y_test = train_test_split(df['Body_3'], y_enc_train, test_size=0.3, shuffle=True)

In [8]:
train.shape

(34997, 10)

In [9]:
test.shape

(15000, 10)

In [16]:
y_train = list(train['Tags'])
y_train

['scala apache-spark distributed-computing k-means bigdata',
 'c# jquery json datatables asp.net -mvc',
 'node.js ssl https openssl tls',
 'rest api postman keycloak put',
 'asp.net jquery asp.net -mvc data-binding drop-down-menu',
 'ruby class module constants class-method',
 'python python .x windows jupyter-notebook python-multiprocessing',
 'java jsf caching jboss pojo',
 'ruby-on-rails ruby activerecord query-optimization constants',
 'php database codeigniter ms-access odbc',
 'angular npm angular-cli fork yarnpkg',
 'javascript date filter google-apps-script filtering',
 'php mysql laravel eloquent laravel',
 'python arrays numpy lstm sliding-window',
 'delphi delphi delphi datasnap delphi-xe',
 'php regex unicode preg-match preg-match-all',
 'java python xls jexcelapi hssf',
 'linux perl process ipc pipe',
 'python django json python serialization',
 'ios iphone objective-c nsdictionary afnetworking',
 'c# combobox telerik telerik-grid telerik-mvc',
 'coldfusion cfml coldfusion

### 1. Approche FIT on TITTLE + BODY and TRANSFORM on TITLE + BODY

In [19]:
vectorizer_1 = CountVectorizer(
    lowercase=False,
    analyzer='word',
    min_df=3,
    max_df= 0.90,
    token_pattern= r'[a-zA_\-+#]*\.?[a-zA_\+#]+',
    )
matrix_train_1 = vectorizer_1.fit_transform(list_title_body_train)

In [20]:
matrix_test_1 = vectorizer_1.transform(list_title_body_test)

In [21]:
vectorizer_1.get_feature_names

<bound method CountVectorizer.get_feature_names of CountVectorizer(lowercase=False, max_df=0.9, min_df=3,
                token_pattern='[a-zA_\\-+#]*\\.?[a-zA_\\+#]+')>

In [31]:
train['body_and_title_2']

38095    experimented partition cardinality k k+ call p...
36016    little experience jquery json great little tut...
40355    parse certificate chain node http request test...
33940    set keycloak server ldap user take advantage s...
41708    achieve effect asp.net mvc jquery got drop lis...
                               ...                        
11284    version corda buildscript ext.corda_release_ve...
44735    ffmpeg android several post theme non build ff...
38161    javascript json jquery easy jsp contain drop l...
860      spirit helpful solved installed redhat suse en...
15796    always unresolved dependency showing android p...
Name: body_and_title_2, Length: 34997, dtype: object

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(
    lowercase=False,
    analyzer='word',
    token_pattern= r'[a-zA_\-+#]*\.?[a-zA_\+#]+'
)
y_train_counts = count_vect.fit_transform(train['Tags'])
y_train_counts.shape

(34997, 15503)

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(
    lowercase=False,
    analyzer='word',
    token_pattern= r'[a-zA_\-+#]*\.?[a-zA_\+#]+'
)
X_train_counts = count_vect.fit_transform(train['body_and_title_2'])
X_train_counts.shape

(34997, 353163)

In [30]:
X_train_counts

<34997x353163 sparse matrix of type '<class 'numpy.int64'>'
	with 2080915 stored elements in Compressed Sparse Row format>

In [18]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(34997, 353163)

In [32]:
X_train_tfidf

<34997x353163 sparse matrix of type '<class 'numpy.float64'>'
	with 2080915 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf,train['Tags'])

In [19]:
sgd = SGDClassifier(random_state=0,loss='log',alpha=0.01,penalty='elasticnet')
lr = LogisticRegression(C=1.0)
svc = SVC(kernel='linear')
nb = MultinomialNB()

In [21]:
y_train

['scala apache-spark distributed-computing k-means bigdata',
 'c# jquery json datatables asp.net -mvc',
 'node.js ssl https openssl tls',
 'rest api postman keycloak put',
 'asp.net jquery asp.net -mvc data-binding drop-down-menu',
 'ruby class module constants class-method',
 'python python .x windows jupyter-notebook python-multiprocessing',
 'java jsf caching jboss pojo',
 'ruby-on-rails ruby activerecord query-optimization constants',
 'php database codeigniter ms-access odbc',
 'angular npm angular-cli fork yarnpkg',
 'javascript date filter google-apps-script filtering',
 'php mysql laravel eloquent laravel',
 'python arrays numpy lstm sliding-window',
 'delphi delphi delphi datasnap delphi-xe',
 'php regex unicode preg-match preg-match-all',
 'java python xls jexcelapi hssf',
 'linux perl process ipc pipe',
 'python django json python serialization',
 'ios iphone objective-c nsdictionary afnetworking',
 'c# combobox telerik telerik-grid telerik-mvc',
 'coldfusion cfml coldfusion

In [22]:
X_train_tfidf

<34997x353163 sparse matrix of type '<class 'numpy.float64'>'
	with 2080915 stored elements in Compressed Sparse Row format>

In [20]:
# One vs Restclassifier
orc_clf = OneVsRestClassifier(estimator=svc).fit(X_train_tfidf, y_train)

KeyboardInterrupt: 

In [1]:
clf

NameError: name 'clf' is not defined

In [78]:
docs_new = [test['body_and_title_2'][0]]
docs_new

['sandbox linux process certain directory process exclusive rw access dir temporary directory e.g python scripting tool directory limiting much functionality process access directory except superusers course sandbox web service basically allows user arbitrary code authorization software process linux user user harm system temporary private directory file protected user webservice giving unix process exclusive rw access directory']

In [ ]:
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, category))

In [17]:
def vectorize(vector, X_train, X_test):
    vector_fit = vector.fit(X_train)

    X_train_vec = vector_fit.transform(X_train)
    X_test_vec = vector_fit.transform(X_test)

    print("Vectorization is completed.")
    return X_train_vec, X_test_vec

def label_encoding(y_train):
    """
        Encode the given list of class labels
        :y_train_enc: returns list of encoded classes
        :labels: actual class labels
    """
    lbl_enc = LabelEncoder()

    y_train_enc = lbl_enc.fit_transform(y_train)
    labels = lbl_enc.classes_

    return y_train_enc, labels

In [18]:
df['Tags']

0                     linux ubuntu process sandbox selinux
1                        java graphics jframe jpanel paint
2          security ssh ssh-keys openssh man-in-the-middle
3               c# winforms sqlite datatable sqlconnection
4                  javascript html node.js mongodb express
                               ...                        
49995    sql-server ssas oledb sql-server-data-tools ve...
49996    python amazon-web-services amazon-s boto pytho...
49997    python machine-learning deep-learning pytorch ...
49998                 java json rest json-patch http-patch
49999    android android-studio android-gradle-plugin g...
Name: Tags, Length: 49997, dtype: object

In [19]:
# Encode the class labels
y_enc_train, labels = label_encoding(df['Tags'])

# Split from the loaded dataset
X_train, X_test, y_train, y_test = train_test_split(df['Body_3'], y_enc_train, test_size=0.3, shuffle=True)

In [20]:
labels

array(['-bit -bit bit bit llvm llvm-gcc',
       '-bit bit bit directshow directshow.net source-filter',
       '-bit makefile -bit autotools multiplatform', ...,
       'zend-framework web-applications dns saas wildcard-subdomain',
       'zend-framework zend-form wrapper containers fieldset',
       'zooming drag shapes konvajs pan'], dtype=object)

In [21]:
# Bag of words (BOW) matrix
bow_vector = CountVectorizer(ngram_range=(1, 1), analyzer='word', max_features=5000, max_df=2, min_df=1)
bow_vector.fit(X_train)

CountVectorizer(max_df=2, max_features=5000)

In [22]:
pipe = Pipeline([('bow', bow_vector),
                 ('tfidf', TfidfTransformer())]).fit(X_train)

In [23]:
train_tfidf = pipe.transform(X_train)
valid_tfidf = pipe.transform(X_test)

In [24]:
print(train_tfidf.shape, valid_tfidf.shape)

(34997, 5000) (15000, 5000)


In [25]:
def lsa_reduction(X_train, X_test, n_comp=120):
    svd = TruncatedSVD(n_components=n_comp)
    normalizer = Normalizer()

    lsa_pipe = Pipeline([('svd', svd),
                         ('normalize', normalizer)]).fit(X_train)

    train_reduced = lsa_pipe.transform(X_train)
    test_reduced = lsa_pipe.transform(X_test)
    return train_reduced, test_reduced

def lsa_nmf_reduction(X_train, X_test, n_comp=120):
    nmf = NMF(n_components=n_comp)
    normalizer = Normalizer()

    lsa_pipe = Pipeline([('nmf', nmf),
                         ('normalize', normalizer)]).fit(X_train)

    train_reduced = lsa_pipe.transform(X_train)
    test_reduced = lsa_pipe.transform(X_test)
    return train_reduced, test_reduced

In [26]:
xtrain_svd, xtest_svd = lsa_reduction(train_tfidf, valid_tfidf, 2000)

# NMF dimensionality function is called only for Multinomial Naive Bayes
# xtrain_svd, xtest_svd = lsa_nmf_reduction(train_tfidf, valid_tfidf)

In [27]:
xtrain_svd

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [28]:
xtest_svd

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# 2. Building model

In [29]:
sgd = SGDClassifier(random_state=0,loss='log',alpha=0.01,penalty='elasticnet')
lr = LogisticRegression(C=1.0)
svc = SVC(kernel='linear')
nb = MultinomialNB()

# One vs Restclassifier
orc_clf = OneVsRestClassifier(estimator=sgd).fit(xtrain_svd, y_train)


KeyboardInterrupt



In [ ]:
print(orc_clf.)

In [28]:
orc_clf.get_params()

{'estimator__C': 1.0, 'estimator__break_ties': False, 'estimator__cache_size': 200, 'estimator__class_weight': None, 'estimator__coef0': 0.0, 'estimator__decision_function_shape': 'ovr', 'estimator__degree': 3, 'estimator__gamma': 'scale', 'estimator__kernel': 'linear', 'estimator__max_iter': -1, 'estimator__probability': False, 'estimator__random_state': None, 'estimator__shrinking': True, 'estimator__tol': 0.001, 'estimator__verbose': False, 'estimator': SVC(kernel='linear'), 'n_jobs': None, 'verbose': 0}


In [29]:
print(orc_clf.__getattribute__())

TypeError: score() missing 2 required positional arguments: 'X' and 'y'

In [30]:
# Predict the test data
y_pred = orc_clf.predict(xtest_svd)

In [31]:
print("Accuracy: %1.3f \tPrecision: %1.3f \tRecall: %1.3f \tF1-Score: %1.3f\n" % (accuracy_score(y_test, y_pred),
                                                                                  precision_score(y_test, y_pred, average='micro'),
                                                                                  recall_score(y_test, y_pred, average='micro'),
                                                                                  f1_score(y_test, y_pred, average='micro')))

Accuracy: 0.001 	Precision: 0.001 	Recall: 0.001 	F1-Score: 0.001



In [ ]:
#jaccard score
# 1/ basic: transforme les donnes avec tfidf - que le titre ou titre + corpus
# 2/ prendre tags uniquement les + freq
#3/ ONEVSCLASSIFIER avec SGD,
#4/ utiliser jaccard score

#4/ tags - mettre sous forme de list

#pour aller + loin:
# effectuer une reduction

In [ ]:
# RFE is popular because it is easy to configure and use and because it is effective at selecting those features (columns) in a training dataset that are more or most relevant in predicting the target variable.
#
# There are two important configuration options when using RFE: the choice in the number of features to select and the choice of the algorithm used to help choose features. Both of these hyperparameters can be explored, although the performance of the method is not strongly dependent on these hyperparameters being configured well.



In [ ]:
#https://machinelearningmastery.com/rfe-feature-selection-in-python/

# explore the algorithm wrapped by RFE
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from matplotlib import pyplot

# get the dataset
def get_dataset():
    X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, random_state=1)
    return X, y

# get a list of models to evaluate
def get_models():
    models = dict()
    # lr
    rfe = RFE(estimator=LogisticRegression(), n_features_to_select=5)
    model = DecisionTreeClassifier()
    models['lr'] = Pipeline(steps=[('s',rfe),('m',model)])
    # perceptron
    rfe = RFE(estimator=Perceptron(), n_features_to_select=5)
    model = DecisionTreeClassifier()
    models['per'] = Pipeline(steps=[('s',rfe),('m',model)])
    # cart
    rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=5)
    model = DecisionTreeClassifier()
    models['cart'] = Pipeline(steps=[('s',rfe),('m',model)])
    # rf
    rfe = RFE(estimator=RandomForestClassifier(), n_features_to_select=5)
    model = DecisionTreeClassifier()
    models['rf'] = Pipeline(steps=[('s',rfe),('m',model)])
    # gbm
    rfe = RFE(estimator=GradientBoostingClassifier(), n_features_to_select=5)
    model = DecisionTreeClassifier()
    models['gbm'] = Pipeline(steps=[('s',rfe),('m',model)])
    return models

# evaluate a give model using cross-validation
def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
    return scores

In [ ]:
# define dataset
X, y = get_dataset()

In [ ]:
# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, X, y)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()